# Breast Cancer Classification with TensorFlow/Keras

This script trains a neural network using TensorFlow/Keras for binary classification.

**Dataset:** Breast Cancer Wisconsin (Diagnostic)
- 569 samples of breast cancer biopsies
- Target: Malignant (1) or Benign (0)

**Features:**
- 30 numeric features computed from digitized images
- Mean, standard error, and worst values of cell nuclei characteristics

%pip install 'tensorflow~=2.15.0' pandas numpy scikit-learn mlflow

In [ ]:
import os

# Reduce TensorFlow logging verbosity
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Disable GPU to prevent memory issues (use CPU only)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Limit TensorFlow threading to prevent resource exhaustion
os.environ['TF_NUM_INTEROP_THREADS'] = '2'
os.environ['TF_NUM_INTRAOP_THREADS'] = '2'

# Now import TensorFlow (after environment variables are set)
import tensorflow as tf

# Additional runtime configuration for memory efficiency
tf.config.threading.set_inter_op_parallelism_threads(2)
tf.config.threading.set_intra_op_parallelism_threads(2)

# Verify TensorFlow is using CPU
print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")
print("TensorFlow configured for CPU-only execution")

# ============================================================================
# Standard imports (after TensorFlow configuration)
# ============================================================================
import argparse
import json
import tempfile
import numpy as np
import pandas as pd
from datetime import datetime

# TensorFlow/Keras imports
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# MLflow imports
import mlflow
import mlflow.tensorflow
from mlflow import set_tracking_uri, set_experiment
from mlflow.client import MlflowClient
from mlflow.models import ModelSignature
from mlflow.types import Schema, TensorSpec

# Scikit-learn imports
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [ ]:
def setup_mlflow(mlflow_uri: str, username: str, password: str) -> MlflowClient:
    """Configure MLflow tracking and return client."""
    os.environ["MLFLOW_TRACKING_USERNAME"] = username
    os.environ["MLFLOW_TRACKING_PASSWORD"] = password
    
    set_tracking_uri(mlflow_uri)
    client = MlflowClient(mlflow_uri)
    
    print(f"MLflow tracking URI: {mlflow_uri}")
    return client


def load_and_prepare_data():
    """Load Breast Cancer dataset and prepare train/test splits."""
    print("\n" + "=" * 80)
    print("LOADING DATASET")
    print("=" * 80)
    
    # Load dataset
    data = load_breast_cancer(as_frame=True)
    X = data.data
    y = data.target
    
    print(f"Dataset: Breast Cancer Wisconsin")
    print(f"Samples: {X.shape[0]:,}")
    print(f"Features: {X.shape[1]}")
    print(f"\nFeature names (first 10):")
    for i, col in enumerate(X.columns[:10], 1):
        print(f"  {i}. {col}")
    print(f"  ... and {X.shape[1] - 10} more")
    
    print(f"\nTarget distribution:")
    print(f"  Malignant (1): {(y == 1).sum()} samples")
    print(f"  Benign (0): {(y == 0).sum()} samples")
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert back to DataFrame to preserve column names
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)
    
    print(f"\nTrain samples: {X_train_scaled.shape[0]:,}")
    print(f"Test samples: {X_test_scaled.shape[0]:,}")
    
    return X_train_scaled, X_test_scaled, y_train, y_test, X.columns.tolist()


def create_model(input_dim: int, hyperparams: dict):
    """Create a lightweight Keras neural network model."""
    # Using a smaller model architecture for memory efficiency
    model = models.Sequential([
        layers.Dense(32, activation='relu', input_shape=(input_dim,)),
        layers.Dropout(hyperparams['dropout']),
        layers.Dense(16, activation='relu'),
        layers.Dropout(hyperparams['dropout']),
        layers.Dense(8, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hyperparams['learning_rate']),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(name='auc')]
    )
    
    return model


def train_model(X_train, y_train, X_test, y_test, hyperparams: dict):
    """Train TensorFlow/Keras model and return predictions."""
    print("\n" + "=" * 80)
    print("TRAINING MODEL")
    print("=" * 80)
    
    print("Hyperparameters:")
    for key, value in hyperparams.items():
        print(f"  {key}: {value}")
    
    # Create model
    model = create_model(X_train.shape[1], hyperparams)
    
    print(f"\nModel architecture:")
    model.summary()
    
    # Early stopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    
    # Train model
    print("\nTraining...")
    history = model.fit(
        X_train, y_train,
        epochs=hyperparams['epochs'],
        batch_size=hyperparams['batch_size'],
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    print("Training completed!")
    
    # Predictions
    y_train_pred_proba = model.predict(X_train).flatten()
    y_test_pred_proba = model.predict(X_test).flatten()
    
    y_train_pred = (y_train_pred_proba > 0.5).astype(int)
    y_test_pred = (y_test_pred_proba > 0.5).astype(int)
    
    return model, y_train_pred, y_test_pred, y_train_pred_proba, y_test_pred_proba


def calculate_metrics(y_true, y_pred, y_pred_proba, dataset_name="Test"):
    """Calculate and return evaluation metrics."""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    
    return {
        f"{dataset_name.lower()}_accuracy": accuracy,
        f"{dataset_name.lower()}_precision": precision,
        f"{dataset_name.lower()}_recall": recall,
        f"{dataset_name.lower()}_f1": f1,
        f"{dataset_name.lower()}_auc": auc
    }


def log_to_mlflow(model, X_train, y_train, X_test, y_test, 
                  y_train_pred, y_test_pred, y_train_pred_proba, y_test_pred_proba, 
                  hyperparams, feature_names):
    """Log model, parameters, and metrics to MLflow using native TensorFlow flavor.
    
    NOTE: This uses native mlflow.tensorflow.save_model() with TensorSpec signature.
    The runtime's SchemaExtractor uses the input_example (DataFrame) to extract
    feature names for /schema, and Model.inference() converts feature dicts to
    ordered arrays using the same feature order.
    """
    print("\n" + "=" * 80)
    print("LOGGING TO MLFLOW")
    print("=" * 80)
    
    # Log hyperparameters
    for key, value in hyperparams.items():
        mlflow.log_param(key, value)
    
    # Calculate and log metrics
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_pred_proba, "Train")
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_pred_proba, "Test")
    all_metrics = {**train_metrics, **test_metrics}
    
    for metric_name, metric_value in all_metrics.items():
        mlflow.log_metric(metric_name, metric_value)
    
    print("\nModel Performance:")
    print(f"  Training Accuracy: {train_metrics['train_accuracy']:.4f}")
    print(f"  Training AUC: {train_metrics['train_auc']:.4f}")
    print(f"  Test Accuracy: {test_metrics['test_accuracy']:.4f}")
    print(f"  Test Precision: {test_metrics['test_precision']:.4f}")
    print(f"  Test Recall: {test_metrics['test_recall']:.4f}")
    print(f"  Test F1: {test_metrics['test_f1']:.4f}")
    print(f"  Test AUC: {test_metrics['test_auc']:.4f}")
    
    # Print confusion matrix
    cm = confusion_matrix(y_test, y_test_pred)
    print(f"\n  Confusion Matrix:")
    print(f"  {cm}")
    
    # Create TensorSpec signature (required by mlflow.tensorflow.save_model)
    # The runtime will use input_example's column names for /schema
    # IMPORTANT: The input tensor name must match Keras's input layer name
    # For Sequential models starting with Dense, this is typically "dense_input"
    num_features = len(feature_names)
    input_spec = TensorSpec(np.dtype(np.float64), (-1, num_features), name="dense_input")
    output_spec = TensorSpec(np.dtype(np.float32), (-1, 1), name="predictions")
    signature = ModelSignature(
        inputs=Schema([input_spec]), 
        outputs=Schema([output_spec])
    )
    
    # IMPORTANT: Use DataFrame as input_example to preserve feature names!
    # The runtime's SchemaExtractor extracts column names from this DataFrame
    # for /schema endpoint and converts feature dicts to arrays for /predict
    input_example = X_test.head(1)  # DataFrame with 30 named columns
    
    print(f"\n  Saving model with TensorSpec signature...")
    print(f"  Input example shape: {input_example.shape}")
    print(f"  Feature names preserved in input_example: {len(input_example.columns)}")
    
    # Save and log model with native TensorFlow flavor
    with tempfile.TemporaryDirectory() as tmpdir:
        local_model_path = os.path.join(tmpdir, "model")
        
        # Save as native TensorFlow model
        mlflow.tensorflow.save_model(
            model,
            local_model_path,
            signature=signature,
            input_example=input_example  # DataFrame preserves column names!
        )
        
        mlflow.log_artifacts(local_model_path, artifact_path="model")
        print("Model artifacts logged successfully!")
        print("  - Using native mlflow.tensorflow flavor")
        print("  - Input example preserves feature names for /schema")
    
    return all_metrics


def create_sample_payload(X_test, y_test, model, feature_names):
    """Create realistic sample prediction payload."""
    # Get a sample
    sample_idx = 0
    sample = X_test.iloc[sample_idx]
    actual_class = y_test.iloc[sample_idx]
    
    # Predict
    predicted_proba = model.predict(sample.values.reshape(1, -1)).flatten()[0]
    predicted_class = int(predicted_proba > 0.5)
    
    return {
        "features": sample.to_dict(),
        "actual_class": int(actual_class),
        "actual_label": "Malignant" if actual_class == 1 else "Benign",
        "predicted_class": predicted_class,
        "predicted_label": "Malignant" if predicted_class == 1 else "Benign",
        "predicted_probability": float(predicted_proba)
    }


def register_model(client: MlflowClient, model_name: str, run_id: str, experiment_id: str):
    """Register model in MLflow Model Registry."""
    print("\n" + "=" * 80)
    print("REGISTERING MODEL")
    print("=" * 80)
    
    model_uri = f"runs:/{run_id}/model"
    
    # Create registered model if it doesn't exist
    try:
        client.get_registered_model(model_name)
        print(f"Model '{model_name}' already exists in registry")
    except Exception:
        try:
            client.create_registered_model(model_name)
            print(f"Created registered model: {model_name}")
        except Exception as e:
            print(f"Could not create registered model: {e}")
    
    # Create model version
    try:
        result = client.create_model_version(
            name=model_name,
            source=model_uri,
            run_id=run_id
        )
        print(f"Model version registered successfully!")
        print(f"   Model Name: {model_name}")
        print(f"   Version: {result.version}")
        print(f"   Run ID: {run_id}")
        return result.version
    except Exception as e:
        print(f"Model registration failed (model still usable via run URI): {e}")
        print(f"   You can deploy using: mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model")
        return None


def print_deployment_info(run_id: str, experiment_id: str, sample_payload: dict):
    """Print deployment instructions and sample payloads."""
    print("\n" + "=" * 80)
    print("TRAINING COMPLETE!")
    print("=" * 80)
    
    print(f"\nRun Information:")
    print(f"  Run ID: {run_id}")
    print(f"  Experiment ID: {experiment_id}")
    print(f"  Model URI: mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model")
    
    print("\n" + "=" * 80)
    print("DEPLOYMENT PAYLOAD (deploy-model API)")
    print("=" * 80)
    
    deploy_payload = {
        "serve_name": "breast-cancer-tensorflow-classifier",
        "model_uri": f"mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model",
        "env": "local",
        "cores": 2,
        "memory": 4,
        "node_capacity": "spot",
        "min_replicas": 1,
        "max_replicas": 3
    }
    
    print(json.dumps(deploy_payload, indent=2))
    
    print("\n" + "=" * 80)
    print("SAMPLE PREDICTION PAYLOAD (first 5 features shown)")
    print("=" * 80)
    
    # Show only first 5 features for brevity
    sample_features = {k: v for i, (k, v) in enumerate(sample_payload["features"].items()) if i < 5}
    sample_features["..."] = "... (30 features total)"
    
    predict_payload = {
        "features": sample_features
    }
    
    print(json.dumps(predict_payload, indent=2))
    
    print(f"\nExpected Output:")
    print(f"  Actual: {sample_payload['actual_label']} (class {sample_payload['actual_class']})")
    print(f"  Model Prediction: {sample_payload['predicted_label']} (class {sample_payload['predicted_class']})")
    print(f"  Probability: {sample_payload['predicted_probability']:.4f}")


In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Train TensorFlow Breast Cancer Classification Model")
    parser.add_argument(
        "--mlflow-uri",
        default="http://darwin-mlflow-lib.darwin.svc.cluster.local:8080",
        help="MLflow tracking URI"
    )
    parser.add_argument(
        "--username",
        default="abc@gmail.com",
        help="MLflow username"
    )
    parser.add_argument(
        "--password",
        default="password",
        help="MLflow password"
    )
    parser.add_argument(
        "--experiment-name",
        default="breast_cancer_tensorflow_classification",
        help="MLflow experiment name"
    )
    parser.add_argument(
        "--model-name",
        default="BreastCancerTensorFlowClassifier",
        help="Registered model name"
    )
    
    args, _ = parser.parse_known_args()
    
    print("\n" + "=" * 80)
    print("BREAST CANCER CLASSIFICATION WITH TENSORFLOW/KERAS")
    print("=" * 80)
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Setup MLflow
    client = setup_mlflow(args.mlflow_uri, args.username, args.password)
    set_experiment(experiment_name=args.experiment_name)
    print(f"Experiment: {args.experiment_name}")
    
    # Load data
    X_train, X_test, y_train, y_test, feature_names = load_and_prepare_data()
    
    # Define hyperparameters
    hyperparams = {
        "epochs": 100,
        "batch_size": 32,
        "learning_rate": 0.001,
        "dropout": 0.3
    }
    
    # Start MLflow run
    with mlflow.start_run(run_name=f"tensorflow_breast_cancer_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
        # Train model
        model, y_train_pred, y_test_pred, y_train_pred_proba, y_test_pred_proba = train_model(
            X_train, y_train, X_test, y_test, hyperparams
        )
        
        # Log to MLflow
        metrics = log_to_mlflow(
            model, X_train, y_train, X_test, y_test,
            y_train_pred, y_test_pred, y_train_pred_proba, y_test_pred_proba,
            hyperparams, feature_names
        )
        
        # Get run information
        run_id = mlflow.active_run().info.run_id
        experiment_id = mlflow.active_run().info.experiment_id
        
        # Create sample payload
        sample_payload = create_sample_payload(X_test, y_test, model, feature_names)
    
    # Register model (outside of run context)
    version = register_model(client, args.model_name, run_id, experiment_id)
    
    # Print deployment information
    print_deployment_info(run_id, experiment_id, sample_payload)
    
    print("\nScript completed successfully!")
    print("=" * 80 + "\n")


if __name__ == "__main__":
    main()